In [3]:
# importing needed libraries
import os
import matplotlib as plt
import numpy as np
from skimage.io import imread
from skimage.transform import resize
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn import svm
from sklearn.metrics import accuracy_score, confusion_matrix
import pickle

In [67]:
# initialised variables needed for processing data
target = [] 
images = [] # Matrix Format
flat_data = [] 
DATADIR = '/content/drive/MyDrive/ML_DATASET'
CATEGORIES = ['apple','banana','mango','orange']

In [ ]:
# preprocessing data
# print(CATEGORIES)
for i in CATEGORIES:
  class_target=CATEGORIES.index(i)
  path=os.path.join(DATADIR,i)
  for img in os.listdir(path):
    img_path=os.path.join(path,img)
    # print(img_path)
    img_array=imread(img_path)
    img_resized = resize(img_array,(150,150,3))
    flat_data.append(img_resized.flatten())
    images.append(img_resized)
    target.append(class_target)
flat_data = np.array(flat_data)
target = np.array(target)
images = np.array(images)

In [69]:
# split data for training and testing
x_train,X_test,y_train,y_test=train_test_split(flat_data,target,test_size=0.3,random_state=101)

In [ ]:
# model building and training
param_grid=[
            {'C':[1,10,100,1000],'kernel':['linear']},
            {'C':[1,10,100,1000],'gamma':[0.001,0.0001],'kernel':['rbf']},
]
svc=svm.SVC(probability=True)
clf=GridSearchCV(svc,param_grid)
clf.fit(x_train,y_train)

In [ ]:
# evaluate model
y_pred= clf.predict(X_test)
accuracy_score(y_pred,y_test)

In [ ]:
confusion_matrix(y_pred,y_test)

In [91]:
# save model
pickle.dump(clf,open('img_model.p','wb'))


In [92]:
#runnig saved model
import pickle
model=pickle.load(open('img_model.p','rb'))

In [ ]:
# testing a image out of the dataset
import os
import matplotlib as plt
import numpy as np
from skimage.io import imread
from skimage.transform import resize
import pickle
model=pickle.load(open('img_model.p','rb'))
flat_data=[]
CATEGORIES = ['apple','banana','mango','orange']
url = input('Enter image url for predication')
img = imread(url)
img_resized = resize(img,(150,150,3))
flat_data.append(img_resized.flatten())
flat_data = np.array(flat_data)
plt.pyplot.imshow(img_resized)
y_out = model.predict(flat_data)
y_out = CATEGORIES[y_out[0]]
print(f' PREDICTED OUTPUT: {y_out}')

In [1]:
# writing python file for webapp
%%writefile app.py
import streamlit as st
import numpy as np
from skimage.io import imread
from skimage.transform import resize
import pickle
from PIL import Image
model=pickle.load(open('img_model.p','rb'))
st.title('Image Classification System using Machine Learning')
st.text('upload a image')
uploaded_file=st.file_uploader("choos an image....", type="jpg")
x=False;
if uploaded_file is not None:
  img=Image.open(uploaded_file)
  st.image(img,caption='uploaded image')
  x=True;
CATEGORIES = ['apple','banana','mango','orange']
if st.button('PREDICT'):
  if x:
    flat_data=[]
    img=np.array(img)
    img_resized = resize(img,(150,150,3))
    flat_data.append(img_resized.flatten())
    flat_data = np.array(flat_data)
    y_out = model.predict(flat_data)
    y_out = CATEGORIES[y_out[0]]
    st.write('result...')
    # st.write(f' PREDICTED OUTPUT: {y_out}')
    q=model.predict_proba(flat_data)
    for index,item in enumerate(CATEGORIES):
      st.write(f'{item} : {q[0][index]*100}%')

Overwriting app.py


In [ ]:
# troubleshoot ngrok connection
# !top #extract ngrok process id;
# !kill -9{<ngrok process-id>}

In [ ]:
# streamlit server intialize
# !streamlit run --server.port 80 app.py >/dev/null 

In [ ]:
# deployment of the model as web app
!pip install streamlit
!pip install pyngrok

In [ ]:
# deploying ngrok server
from pyngrok import ngrok
!nohup streamlit run --server.port 80 app.py >/dev/null &
url=ngrok.connect(port='80')
url